In [7]:
## Fetch

import requests
import json
from project_secrets import nyt_secrets

year = 1963
month = 6
apiKey = nyt_secrets["API_KEY"]

requestUrl = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apiKey}"

print(requestUrl)
res = requests.get(requestUrl)
obj = json.loads(res.text)

https://api.nytimes.com/svc/archive/v1/1963/6.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


In [8]:
## Parse article

from multiprocessing import Pool, Manager
from tqdm.notebook import tqdm
from modules.lang_utils import parseArticle

docs = obj["response"]["docs"]

article_word_count = []

with Manager() as smm:
    sl = smm.list()
    sl.extend(docs)
    with Pool(processes=10) as p:
        with tqdm(total=len(sl), desc="Going through articles") as pbar:
            for res in p.imap_unordered(parseArticle, sl, chunksize=10):
                article_word_count.append(res)
                pbar.update()

Going through articles:   0%|          | 0/12177 [00:00<?, ?it/s]

In [9]:
## Save results to file for testing

# with open(f"./testfiles/result_{year}_{month}.json", "w", encoding="utf-8") as json_file:
#    json_file.write(json.dumps(article_word_count, indent="\t"))

In [10]:
import mysql.connector
from project_secrets import db_secrets
from modules.db_utils import create_article_in_db

db_host = db_secrets["HOST"]
db_user = db_secrets["USER"]
db_password = db_secrets["PASSWORD"]
db_database = "udocloud"

mydb = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_database
)

for article in article_word_count:
    create_article_in_db(article,mydb)

ImportError: cannot import name 'create_article_in_db' from 'modules.db_utils' (c:\Users\Matth\UdosWordCloud\Scraper\modules\db_utils.py)